# RNN을 이용한 텍스트 생성


    '경마장에 있는 말이 뛰고 있다'
    '그의 말이 법이다'
    '가는 말이 고와야 오는 말이 곱다'
<pre> 요고도 가능
</pre>

In [1]:
import numpy as np 
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [11]:
text = '''경마장에 있는 말이 뛰고 있다.
그의 말이 법이다.
가는 말이 고와야 오는 말이 곱다.'''

In [12]:
# print(text)

- 텍스트 전처리 영어처럼 스페이스를 기준

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [14]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts([text])

In [16]:
# 특징 
# 1. word_index 순서가 가나다 순서가 아니고, 많이 나온 단어 순
# 2. 0 index가 없다. 1번부터 시작한다.
t.word_index

{'가는': 8,
 '경마장에': 2,
 '고와야': 9,
 '곱다': 11,
 '그의': 6,
 '뛰고': 4,
 '말이': 1,
 '법이다': 7,
 '오는': 10,
 '있는': 3,
 '있다': 5}

In [18]:
# 단어 집합 크기 - keras Tokenizer의 시작 인덱스가 1이기 때문
# 케라스 토크나이저 쓸때는 늘 보캡사이즈에 1을 더해서 사용하자
vocab_size = len(t.word_index) + 1
vocab_size

12

In [20]:
t.texts_to_sequences(['그의 말이 법이다'])

[[6, 1, 7]]

In [22]:
# 학습에 사용될 샘플 시퀀스 
sequences = []

for line in text.split('\n'):
    encoded=t.texts_to_sequences([line])[0]
    print(encoded)

[2, 3, 1, 4, 5]
[6, 1, 7]
[8, 1, 9, 10, 1, 11]


In [24]:
sequences = []
for line in text.split('\n'):
    encoded=t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequences.append(encoded[:i+1])

sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [28]:
# 만든 리스트 중에 제일 긴놈이 뭐야? => max_len
max_len = max(len(seq) for seq in sequences)
max_len

6

### 패딩 두두등장.

 - Padding

        전체 샘플길이를 max_len에 맞추도록 0을 추가
        pre 옵션을 주면 앞쪽을 채움

In [29]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences
# 아래는 RNN의 쉐입과 관련이 있어. 타입 스텝과, 인풋디멘션(피쳐개수)
# 맨 마지막 컬럼 3,1,4,5,~ 가 y값이 되는것. 그 앞은 x값이 될것 

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]], dtype=int32)

In [30]:
X = sequences[:, :-1]
y = sequences[:, -1]

In [31]:
X.shape  # 시퀀스 개수 11개 타임스텝 5개 인풋디멘전은 단어의 개수가 보캡사이즈 만큼. 

(11, 5)

In [33]:
Y = to_categorical(y)
Y
# 순서좀 맞춰주면 안되나 컴퓨터야?

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

## 모델 정의
- Embedding
- SimpleRNN
- Dense

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [44]:
# Embedding=4, SimpleRNN=32
model = Sequential([
    Embedding(vocab_size, 4, input_length=max_len-1),      # 12, 4, 5 인셈이다. 
    SimpleRNN(32),  # 기본 액티베이션 tanh 로 되어있어.  # 왜 RNN에 인풋쉐입 및 타임스텝 안주느냐. 위에 준놈이랑 동일 그래로 받아쓰기 때문. 
                    # RNN 층 의 인풋디멘젼은 4(임베딩) 이며, 타임스텝스는 maxlen-1 dls 5다 위에 다 나와있어서 안쓴다.
    Dense(vocab_size, activation='softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 5, 4)              48        
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 32)                1184      
                                                                 
 dense_2 (Dense)             (None, 12)                396       
                                                                 
Total params: 1,628
Trainable params: 1,628
Non-trainable params: 0
_________________________________________________________________


## 모델 설정 /학습

In [46]:
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
)

In [47]:
hist = model.fit(X, Y, epochs=200, verbose=0)

In [50]:
# 정확도 100%
hist.history['accuracy'][-1]  # 학습된 제일 마지막 것. 

0.9090909361839294

In [51]:
from google.colab import files
up = files.upload()

In [52]:
from my_util import sentence_generation

In [54]:
# 모델, 토크나이저, max_len, 현재 단어, 반복할 횟수
sentence_generation(model, t, max_len, '경마장에', 3)

'경마장에 말이 말이 뛰고'

해석은 경마장에 를 입력하면, 우리가 기대할 수 있는 아웃풋은 

In [61]:
# 모델, 토크나이저, max_len, 현재 단어, 반복할 횟수  #?????
sentence_generation(model, t, max_len, '경마장에', 12)

'경마장에 말이 말이 뛰고 있다 곱다 말이 있다 말이 곱다 곱다 말이 곱다'

In [62]:
# 모델, 토크나이저, max_len, 현재 단어, 반복할 횟수
sentence_generation(model, t, max_len, '가는', 5)

'가는 말이 고와야 오는 말이 곱다'

### 모델 변화
- Embedding vecter 갯수 2, 4, 6
- RNN 유닛 개수 24, 32, 48 

In [65]:
for n_embed in [2,4,6]:
    for n_unit in [24,32,48]:
        model = Sequential([
                    Embedding(vocab_size, n_embed, input_length=max_len-1),
                    SimpleRNN(n_unit),
                    Dense(vocab_size, activation='softmax')
        ])
        model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
        hist = model.fit(X, Y, epochs=200, verbose=0)
        print('====================================')
        print(f"Embedding vector: {n_embed}, RNN unit: {n_unit}")
        print(f"정확도:{hist.history['accuracy'][-1]:.4f}")
        print(sentence_generation(model, t, max_len, '경마장에', 4))
        print(sentence_generation(model, t, max_len, '그의', 2))
        print(sentence_generation(model, t, max_len, '가는', 5))
        print()

Embedding vector: 2, RNN unit: 24
정확도:0.8182
경마장에 말이 말이 뛰고 있다
그의 말이 말이
가는 말이 고와야 오는 말이 곱다

Embedding vector: 2, RNN unit: 32
정확도:0.9091
경마장에 말이 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector: 2, RNN unit: 48
정확도:1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector: 4, RNN unit: 24
정확도:0.9091
경마장에 말이 고와야 오는 말이
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector: 4, RNN unit: 32
정확도:0.8182
경마장에 말이 말이 오는 말이
그의 말이 법이다
가는 말이 법이다 오는 말이 곱다

Embedding vector: 4, RNN unit: 48
정확도:1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector: 6, RNN unit: 24
정확도:0.9091
경마장에 말이 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector: 6, RNN unit: 32
정확도:1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector: 6, RNN unit: 48
정확도:1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

